#### CSE 158R / Assignment 1
Harshita Saha

#### Setup

Importing packages

In [1]:
import gzip
from collections import defaultdict
import math
import scipy.optimize
from sklearn import svm
import numpy
import string
import random
import string
from sklearn import linear_model
from nltk.corpus import stopwords
from nltk.stem.porter import *

In [2]:
import warnings
warnings.filterwarnings("ignore")

Defining basic functions

In [3]:
def assertFloat(x):
    assert type(float(x)) == float

def assertFloatList(items, N):
    assert len(items) == N
    assert [type(float(x)) for x in items] == [float]*N

In [4]:
def readGz(path):
    for l in gzip.open(path, 'rt'):
        yield eval(l)

In [5]:
def readCSV(path):
    f = gzip.open(path, 'rt')
    f.readline()
    for l in f:
        u,b,r = l.strip().split(',')
        r = int(r)
        yield u,b,r

#### Some data structures that will be useful

Complete dataset

In [6]:
allRatings = []
for l in readCSV("train_Interactions.csv.gz"):
    allRatings.append(l)

In [7]:
len(allRatings)

200000

Splitting data into training and validation set, and creating dictionaries on Training set

In [8]:
ratingsTrain = allRatings[:190000]
ratingsValid = allRatings[190000:]

ratingsPerUser = defaultdict(list)
ratingsPerItem = defaultdict(list)

for u,b,r in ratingsTrain:
    
    # dictionary of users and the books they read with user ratings
    ratingsPerUser[u].append((b,r))
    
    # dictionary of books and the users that read them with user ratings
    ratingsPerItem[b].append((u,r))

#### Task 1: Read prediction    

List of popular books, and their counts (mostPopular), and set of top 65% of read books (return1)

In [48]:
# copied from baseline code
bookCount = defaultdict(int)
totalRead = 0

for user,book,_ in readCSV("train_Interactions.csv.gz"):
    bookCount[book] += 1
    totalRead += 1

mostPopular = [(bookCount[x], x) for x in bookCount]
mostPopular.sort()
mostPopular.reverse()

return1 = set()
count = 0

# finding the top 65% of the most read books 
for ic, i in mostPopular:
    count += ic
    return1.add(i)
    if count > int(totalRead*0.65): break

Creating data structures for validation set and adding negative pairs

In [10]:
ratingsPerUserV = defaultdict(list)
ratingsPerItemV = defaultdict(list)

# dictionary of users and the books they read with user ratings
for u,b,r in ratingsValid:
    ratingsPerUserV[u].append((b,r))
    
# dictionary of books and the users that read them with user ratings
for u,b,r in allRatings:
    ratingsPerItemV[b].append((u,r))

In [11]:
# all books in validation set
bookIds = list(ratingsPerItemV.keys())

# adding negative pairs (user, unread_book) to validation set
for user in ratingsPerUserV.keys():
    read_books = [i[0] for i in ratingsPerUserV[user]]
    unread_book = numpy.random.choice([b for b in bookIds if b not in read_books])
    ratingsValid.append((user, unread_book))

In [52]:
[i for i in ratingsValid if i[0] == 'u74452117']

[('u74452117', 'b44922252', 3), ('u74452117', 'b66688293')]

#### Creating helper functions for prediction

Jaccard related functions and inital calculation for threshold of jacc_user

In [12]:
def Jaccard(s1, s2):
    numer = len(s1.intersection(s2))
    denom = len(s1.union(s2))
    if denom == 0:
        return 0
    return numer / denom

In [13]:
def jacc_user(u, i):
    bprime = [j[0] for j in ratingsPerUser[u]]
    users_read_b = [j[0] for j in ratingsPerItem[i]]
    
    jaccard_list = numpy.array([])
    for b in bprime:
        users_read_bprime = [j[0] for j in ratingsPerItem[b]]
        jaccard = Jaccard(set(users_read_b), set(users_read_bprime))
        jaccard_list = numpy.append(jaccard_list, jaccard)
    
    return jaccard_list

In [54]:
ratingsPerUser['u67805239']

[('b61372131', 4),
 ('b09049778', 0),
 ('b66416013', 3),
 ('b13061166', 0),
 ('b03047230', 0),
 ('b11179371', 0),
 ('b30494145', 0),
 ('b66908318', 0),
 ('b95800909', 0),
 ('b79619792', 0),
 ('b88947899', 4),
 ('b08004121', 4),
 ('b76040943', 0),
 ('b09589765', 0),
 ('b79253970', 4),
 ('b26847239', 0),
 ('b79098002', 0),
 ('b22348421', 0)]

In [56]:
ratingsPerItem['b61372131']

[('u67805239', 4),
 ('u42487937', 2),
 ('u73059201', 4),
 ('u06042879', 3),
 ('u98143812', 4),
 ('u24081627', 5),
 ('u92909378', 3),
 ('u78225145', 2),
 ('u41155507', 3),
 ('u89709107', 2),
 ('u46593302', 4),
 ('u03476667', 5),
 ('u15273524', 4),
 ('u46965119', 3),
 ('u51751023', 5),
 ('u49766428', 4),
 ('u48089762', 4),
 ('u01071398', 2),
 ('u33561653', 3),
 ('u99811205', 0),
 ('u92511535', 3),
 ('u03379358', 4),
 ('u28242564', 5),
 ('u33896303', 3),
 ('u11915055', 1),
 ('u72002179', 1),
 ('u67652911', 4),
 ('u22497171', 4),
 ('u09299901', 5),
 ('u95030396', 5),
 ('u87998402', 2),
 ('u84517883', 4),
 ('u96564934', 3),
 ('u92595296', 1),
 ('u21056033', 4),
 ('u45796950', 2),
 ('u38301742', 3),
 ('u54426184', 1),
 ('u90105683', 2),
 ('u55760432', 4),
 ('u22387175', 3),
 ('u45944018', 4),
 ('u53021409', 5),
 ('u67181585', 5),
 ('u95278557', 3),
 ('u62772540', 5),
 ('u28332618', 2),
 ('u07301204', 3),
 ('u81545593', 4),
 ('u08830362', 3),
 ('u17968157', 1),
 ('u93476362', 3),
 ('u49555043

In [14]:
def jacc_item(u, i):
    cprime = [j[0] for j in ratingsPerItem[i]]
    items_read_c = [j[0] for j in ratingsPerUser[u]]
    
    jaccard_list = numpy.array([])
    for c in cprime:
        items_read_cprime = [j[0] for j in ratingsPerUser[c]]
        jaccard = Jaccard(set(items_read_c), set(items_read_cprime))
        jaccard_list = numpy.append(jaccard_list, jaccard)
    
    return jaccard_list

Creating threshold for Jaccard similarity

In [15]:
jaccard_list_u = numpy.array([])

for i in ratingsValid:
    user = i[0]
    book = i[1]
    jaccard_list_u = numpy.append(jaccard_list_u, jacc_user(user, book))

In [16]:
threshold_u = numpy.percentile(jaccard_list_u, 85)
threshold_u # 0.0035087719298245615

0.0035211267605633804

Popularity related function and dictionary of book IDs and corresponding reads

In [17]:
dict_books = [i[1] for i in mostPopular]
dict_vals = [i[0] for i in mostPopular]
dict_pop = {dict_books[i]: dict_vals[i] for i in range(len(dict_books))}

In [18]:
def popularity(book): 
    if book in dict_pop.keys():
        return dict_pop[book]
    else:
        return 0

#### Starting to build up to predictions

Defining feature function

In [59]:
ratingsValid[-10]

('u40016204', 'b23590291')

In [60]:
ratingsValid[10]

('u16245954', 'b51242397', 5)

In [61]:
[i for i in ratingsValid if i[0] == 'u16245954']

[('u16245954', 'b51242397', 5), ('u16245954', 'b36794167')]

In [19]:
def feat(u, i):
    pop = popularity(i) 
    jacc_u = jacc_user(u, i)
    jacc_i = jacc_item(u, i)
    if len(jacc_u) == 0:
        jacc_u = 0
    if len(jacc_i) == 0:
        jacc_i = 0
    return [1] + [pop, numpy.max(jacc_u), numpy.max(jacc_i)]

Creating list of feature values for validation set

In [20]:
X1 = [feat(d[0], d[1]) for d in ratingsValid] 
[len(i) for i in X1[:10]]

[4, 4, 4, 4, 4, 4, 4, 4, 4, 4]

Creating list of true read values for validation set

In [21]:
y_read = [1]*len(ratingsValid[:10000])
y_unread = [0]*len(ratingsValid[10000:])

y1 = numpy.concatenate([y_read, y_unread])

Conducting logistic regression and generating model

In [22]:
mod1 = linear_model.LogisticRegression(C=1, class_weight = 'balanced')
mod1.fit(X1, y1)

LogisticRegression(C=1, class_weight='balanced')

Generating effective threshold for Jaccard similarity of jacc_user

In [23]:
threshold_u # 0.0035087719298245615
threshold_u*0.65 # 0.002280701754385965

0.002288732394366197

Finding the number of times last book in set return1 was read

In [24]:
# copied from baseline code
return_read = []
count_read = 0

# finding the top 65% of the most read books 
for ic, i in mostPopular:
    count_read += ic
    return_read.append(i)
    if count_read > int(totalRead*0.65): break

In [25]:
return_read[-1] # 'b19310775'

'b19310775'

In [26]:
len(return_read) == numpy.sum([1 for i in return_read if i in return1])

True

In [27]:
len(return_read) == len(return1)

True

In [28]:
top_65_count = dict_pop[return_read[-1]]
top_65_count # 44

44

#### Defining predictor function

In [29]:
def predictor(user, book): # 0.7612 on seen data in leaderboard
    
    # books read by validation user in training set
    bprime = [j[0] for j in ratingsPerUser[user]]
    
    # if no books read by user in training set
    if len(bprime) == 0:
        # predicting based on if book is in the top 65% most read books
        if book in return1: 
            return 1
        elif popularity(book) >= top_65_count:
            return 1
        
        # predicting based on logistic regressor
        else:
            X = [feat(user, book)] 
            return mod1.predict(X)[0]
    
    # if books read by user in training set
    # predicting based on jacc_user threshold or if book is in the top 65% most read books
    if numpy.mean(jacc_user(user, book)) >= threshold_u*0.65 or book in return1: 
        return 1
    elif popularity(book) >= top_65_count:
        return 1
    
    # predicting based on logistic regressor
    else:
        X1 = [feat(user, book)] 
        return mod1.predict(X1)[0]

#### Writing read predictions to file

In [30]:
predictions = open("predictions_Read.csv", 'w')
for l in open("pairs_Read.csv"):
    if l.startswith("userID"):
        predictions.write(l)
        continue
    u,b = l.strip().split(',')
    prediction = str(predictor(u, b))
    predictions.write(u + "," + b + "," + prediction + "\n")

predictions.close()

#### Task 2: Category prediction

#### Setup

In [31]:
data = []

for d in readGz("train_Category.json.gz"):
    data.append(d)

Creating list of stopwords to exclude in generating prediction

In [32]:
stop = stopwords.words('english')

Generating dictionary of words without punctuation, and their counts, for the dataset, while ignoring stop words 

In [33]:
wordCount = defaultdict(int)
punctuation = set(string.punctuation)

# removing punctuation from words and recording occurences into dictionary
for d in data:
    r = ''.join([c for c in d['review_text'].lower() if not c in punctuation])
    for w in r.split():
        if w not in stop:
            wordCount[w] += 1

In [34]:
counts = [(wordCount[w], w) for w in wordCount]
counts.sort()
counts.reverse()

In [35]:
counts[:10]

[(145127, 'book'),
 (69804, 'read'),
 (66795, 'story'),
 (61712, 'one'),
 (55357, 'like'),
 (47035, 'really'),
 (40546, 'characters'),
 (37565, 'series'),
 (36309, 'love'),
 (31860, 'first')]

Generating list, set, and dictionary of size 7000

In [36]:
words = [x[1] for x in counts[:7000]]
wordId = dict(zip(words, range(len(words))))
wordSet = set(words)

#### Starting to build up to predictions

Defining feature function 

In [37]:
def feature(datum):
    feat = [0]*len(words)
    r = ''.join([c for c in datum['review_text'].lower() if not c in punctuation])
    for w in r.split():
        if w in wordSet:
            feat[wordId[w]] += 1
    feat.append(1) # offset
    return feat

Generating feature list and true category list for data

In [38]:
X2 = [feature(d) for d in data]
y2 = [d['genreID'] for d in data]

Splitting X2 and y2 into training and validation sets

In [39]:
Xtrain = X2[:9*len(X2)//10]
ytrain = y2[:9*len(y2)//10]
Xvalid = X2[9*len(X2)//10:]
yvalid = y2[9*len(y2)//10:]

#### Generating prediction model

In [40]:
# 0.7222 on seen data in leaderboard
mod2 = linear_model.LogisticRegression(C=1) 
mod2.fit(Xtrain,ytrain)

LogisticRegression(C=1)

#### Writing category predictions to file

In [41]:
data_test = []

for d in readGz("test_Category.json.gz"):
    data_test.append(d)

In [42]:
Xtest = [feature(d) for d in data_test]
pred_test = mod2.predict(Xtest)

In [43]:
len(feature(data_test[0]))

7001

In [44]:
predictions = open("predictions_Category.csv", 'w')
pos = 0

for l in open("pairs_Category.csv"):
    if l.startswith("userID"):
        predictions.write(l)
        continue
    u,b = l.strip().split(',')
    _ = predictions.write(u + ',' + b + ',' + str(pred_test[pos]) + '\n')
    pos += 1

predictions.close()